In [1]:
from ISR.models import RRDN
from ISR.models import Discriminator
from ISR.models import Cut_VGG19
import os

Using TensorFlow backend.


In [2]:
lr_train_patch_size = 40
layers_to_extract = [5, 9]
scale = 2
hr_train_patch_size = lr_train_patch_size * scale

rrdn  = RRDN(arch_params={'C':4, 'D':3, 'G':64, 'G0':64, 'T':10, 'x':scale}, patch_size=lr_train_patch_size)
f_ext = Cut_VGG19(patch_size=hr_train_patch_size, layers_to_extract=layers_to_extract)
discr = Discriminator(patch_size=hr_train_patch_size, kernel_size=3)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [5]:
from ISR.train import Trainer
loss_weights = {
  'generator': 0.01,
  'feature_extractor': 1.0,
  'discriminator': 0.005
}
losses = {
  'generator': 'mae',
  'feature_extractor': 'mse',
  'discriminator': 'binary_crossentropy'
} 

log_dirs = {'logs': './logs', 'weights': './weights'}

learning_rate = {'initial_value': 0.0004, 'decay_factor': 0.5, 'decay_frequency': 30} 

flatness = {'min': 0.0, 'max': 0.0, 'increase': 0.0, 'increase_frequency': 0}

base_path = '/home/canrobins13/landsat_8'

trainer = Trainer(
    generator=rrdn,
    discriminator= discr,
    feature_extractor=f_ext,
    lr_train_dir=os.path.join(base_path, 'LR_2x', 'train'),
    hr_train_dir=os.path.join(base_path, 'HR_2x', 'train'),
    lr_valid_dir=os.path.join(base_path, 'LR_2x', 'valid'),
    hr_valid_dir=os.path.join(base_path, 'HR_2x', 'valid'),
    loss_weights=loss_weights,
    learning_rate=learning_rate,
    flatness=flatness,
    dataname='div2k',
    log_dirs=log_dirs,
    weights_generator='weights/rrdn-C4-D3-G64-G064-T10-x2/2019-12-06_0633/rrdn-C4-D3-G64-G064-T10-x2_epoch204.hdf5',
    weights_discriminator=None,
    fallback_save_every_n_epochs=2,
    n_validation=1,
    metrics={'generator': 'PSNR'}
)

In [ ]:
trainer.train(
    epochs=500,
    steps_per_epoch=100,
    batch_size=4,
    monitored_metrics={'val_generator_PSNR': 'max'}
)


Training details:
  generator: 
    name: rrdn
    parameters: {'x': 2, 'G': 64, 'G0': 64, 'C': 4, 'D': 3, 'T': 10}
    weights_generator: weights/rrdn-C4-D3-G64-G064-T10-x2/2019-12-06_0633/rrdn-C4-D3-G64-G064-T10-x2_epoch204.hdf5
  discriminator: 
    name: srgan-large
    weights_discriminator: None
  training_parameters: 
    loss_weights: {'generator': 0.01, 'discriminator': 0.005, 'feature_extractor': 1.0}
    hr_valid_dir: /home/canrobins13/landsat_8/HR_2x/valid
    log_dirs: {'logs': './logs', 'weights': './weights'}
    lr_valid_dir: /home/canrobins13/landsat_8/LR_2x/valid
    dataname: div2k
    adam_optimizer: {'epsilon': 1e-07, 'beta2': 0.999, 'beta1': 0.9}
    losses: {'generator': 'mae', 'discriminator': 'binary_crossentropy', 'feature_extractor': 'mse'}
    batch_size: 4
    learning_rate: {'initial_value': 0.0004, 'decay_frequency': 30, 'decay_factor': 0.5}
    lr_train_dir: /home/canrobins13/landsat_8/LR_2x/train
    starting_epoch: 204
    lr_patch_size: 40
    steps_

4/4 [==============================] - 4s 1s/step


{'train_d_real_loss': 0.2061103, 'val_feature_extractor_loss': 0.010587015189230442, 'train_loss': 0.03119396, 'train_feature_extractor_loss': 0.02790272, 'val_generator_PSNR': 49.38444137573242, 'val_loss': 0.010403688997030258, 'train_d_real_accuracy': 0.0, 'val_discriminator_loss': 0.6635745763778687, 'train_d_fake_accuracy': 0.0, 'val_generator_loss': 0.0024662124924361706, 'train_d_fake_loss': 0.20823258, 'train_generator_loss': 0.0026941025, 'train_generator_PSNR': 48.19935, 'train_discriminator_loss': 2.8202355}
val_generator_PSNR improved from       -inf to   49.38444
Saving weights
Epoch 205/500
Current learning rate: 6.24999984211172e-06

100%|██████████| 100/100 [03:03<00:00,  1.84s/it]
Epoch 205 took      183.7s


4/4 [==============================] - 0s 47ms/step


{'train_d_real_loss': 0.20301677, 'val_feature_extractor_loss': 0.011767996475100517, 'train_loss': 0.02798843, 'train_feature_extractor_loss': 0.019920493, 'val_generator_PSNR': 48.99063491821289, 'val_loss': 0.011098388582468033, 'train_d_real_accuracy': 0.0, 'val_discriminator_loss': 0.6484482288360596, 'train_d_fake_accuracy': 0.0, 'val_generator_loss': 0.0025342467706650496, 'train_d_fake_loss': 0.21588631, 'train_generator_loss': 0.0029919602, 'train_generator_PSNR': 46.814777, 'train_discriminator_loss': 2.7158923}
val_generator_PSNR did not improve.
Epoch 206/500
Current learning rate: 6.24999984211172e-06

100%|██████████| 100/100 [02:58<00:00,  1.78s/it]
Epoch 206 took      178.4s


...

{'train_d_real_loss': 0.20088184, 'val_feature_extractor_loss': 0.0095561183989048, 'train_loss': 0.026976192, 'train_feature_extractor_loss': 0.02045698, 'val_generator_PSNR': 49.118709564208984, 'val_loss': 0.008004792034626007, 'train_d_real_accuracy': 0.0, 'val_discriminator_loss': 0.31779634952545166, 'train_d_fake_accuracy': 0.0, 'val_generator_loss': 0.002532052341848612, 'train_d_fake_loss': 0.20144093, 'train_generator_loss': 0.0028908954, 'train_generator_PSNR': 47.706005, 'train_discriminator_loss': 2.8225257}
val_generator_PSNR did not improve.
Epoch 271/500
Current learning rate: 7.81249980263965e-07

100%|██████████| 100/100 [02:59<00:00,  1.80s/it]
Epoch 271 took      179.6s


4/4 [==============================] - 0s 47ms/step


{'train_d_real_loss': 0.20185684, 'val_feature_extractor_loss': 0.009373759850859642, 'train_loss': 0.02048307, 'train_feature_extractor_loss': 0.008807965, 'val_generator_PSNR': 49.1560173034668, 'val_loss': 0.00803653709590435, 'train_d_real_accuracy': 0.0, 'val_discriminator_loss': 0.34256622195243835, 'train_d_fake_accuracy': 0.0, 'val_generator_loss': 0.0025098221376538277, 'train_d_fake_loss': 0.20204107, 'train_generator_loss': 0.0025368878, 'train_generator_PSNR': 48.95303, 'train_discriminator_loss': 2.8568954}
val_generator_PSNR did not improve.
Epoch 272/500
Current learning rate: 7.81249980263965e-07

100%|██████████| 100/100 [03:00<00:00,  1.81s/it]
Epoch 272 took      180.6s


4/4 [==============================] - 0s 47ms/step


{'train_d_real_loss': 0.20114192, 'val_feature_extractor_loss': 0.009628103114664555, 'train_loss': 0.028285626, 'train_feature_extractor_loss': 0.021392925, 'val_generator_PSNR': 48.988189697265625, 'val_loss': 0.008377952501177788, 'train_d_real_accuracy': 0.0, 'val_discriminator_loss': 0.3839941918849945, 'train_d_fake_accuracy': 0.0, 'val_generator_loss': 0.002598102670162916, 'train_d_fake_loss': 0.20124847, 'train_generator_loss': 0.0031984365, 'train_generator_PSNR': 46.795532, 'train_discriminator_loss': 2.8381605}
val_generator_PSNR did not improve.
Saving weights anyways.
Epoch 273/500
Current learning rate: 7.81249980263965e-07

 71%|███████   | 71/100 [02:18<00:52,  1.81s/it]